# Google Search Console API
In diesem Notebook:
- Abfrage der Google Search Console API.
- Eigentlich aufwändiger, dank des Package ["searchconsole"](https://github.com/joshcarty/google-searchconsole) stark vereinfacht. (Josh Carty)
- Simple Authentifizierung des Google Accounts, ebenfalls dank dieses Package.

## Zuerst benötigt ihr ein Projekt in der Google Cloud
→ https://console.cloud.google.com/home/dashboard

![api keys](data/g_api_key.jpg)

## Authentifizierung
Wenn ihr die `client_secret.json` Datei abgespeichert habt, könnt ihr sie in der `.authenticate` Funktion angeben.
```python
# Authentifizierung
searchconsole.authenticate(client_config='client_secret.json')

# Authentifizierung & Speichern der Credentials
searchconsole.authenticate(client_config='client_secret.json', serialize='path/to/cred_file')

# Authentifizierung mit Credentials
searchconsole.authenticate(client_config='client_secret.json', credentials='path/to/cred_file')
```

In [ ]:
import searchconsole

In [ ]:
account = searchconsole.authenticate(client_config='client_secret.json', credentials='credentials.json')

In [ ]:
webproperty = account['https://www.serienjunkies.de/']

## Abfragen
Die Abfragen können mittels `method-chaining` an `webproperty.query` angefügt werden.

In [ ]:
report = webproperty.query.range('today', days=-7).dimension('date').get()

In [ ]:
report

In [ ]:
report.raw

In [ ]:
report.rows

In [ ]:
report.rows[0].date

In [ ]:
for row in report.rows:
    print(f'{row.date} - clicks [{row.clicks}] - impressions [{row.impressions}]')

# Aber wir wollen ja mit DataFrames arbeiten. 
# 🚀 Go Pandas, go!

In [ ]:
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
# %load -r 33:38 helpers/helpers.py
sns.set(context='talk',
        rc={'figure.figsize':(15,6),
            'axes.titlepad':18,
            'axes.titlesize':22,
            'figure.constrained_layout.use':True})

## Abfrage für alle Monate
Dank des Package kann mittels `.range('today', months=-16)` sehr simple ein größerer Umfang heruntergeladen werden. Dabei werden alle paginierten Seiten der API geladen. Mittels der Funktion `.to_dataframe()` wird das Ergebnis der API direkt in einen DataFrame überführt.

In [ ]:
report = webproperty.query.range('today', months=-16).dimension('date').get()
df_totals = report.to_dataframe()
df_totals['date'] = pd.to_datetime(df_totals['date'])
df_totals.head()

## Abfrage mit mehreren Dimensionen
Auch die Kombination von Dimensionen ist einfach. Alle zugelassenen Kombinationen werden so angegeben `.dimension('date','page','query')`. 

**Achtung:** `searchAppearance` kann mit keiner anderen Dimension kombiniert werden!

In [ ]:
sa = webproperty.query.range('today', months=-1).dimension('searchAppearance').get().to_dataframe()
sa.head()

In [ ]:
df = webproperty.query.range('today', days=-9).dimension('date','page','query').get().to_dataframe()
df['date'] = pd.to_datetime(df.date)
df.head()

## Zeit für einen ersten Plot
In den Beispielen haben wir einmal die Gesamtwerte erfasst, aber auch die Daten mit Suchanfragen und Seiten. Zeigen wir beide Zeitverläufe in einer Grafik wird deutlich, dass die Summen (wie wir ja alle schon länger wissen) der Daten die Suchanfragen und Seiten enthalten, niemals die Gesamtwerte ergeben.

In [ ]:
ax = df_totals.query('date >= @df.date.min()').groupby('date')['clicks'].sum().plot(label='totals')
ax = df.groupby('date')['clicks'].sum().plot(label='page&query', ax=ax)
ax.legend()

## Hinzufügen weiterer Features
Im Laufe einer Analyse ist es so gut wie immer nötig, weitere Spalten an das Datenset anzufügen. Das kann natürlich ein anderes Datenset sein, welches mittels `.join` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) oder `.merge` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) angefügt wird. In unserem einfachen Beispiel errechnen wir aus den Daten eine `brand`-Spalte und eine `word_count`-Spalte.

In [ ]:
df['brand'] = df['query'].str.contains('serienjunkies')
df.head()

### Die %%timeit cell-magic
Neben anderen nützlichen `cell-magic` Funktionen, könnt ihr mit `%%timeit` testen, wie schnell euer Code ausgefüht wird.

In [ ]:
%%timeit
df['word_count'] = df['query'].str.split(r'\s').str.len().head()

In [ ]:
%%timeit
df['word_count'] = [len(x.split(r'\s')) for x in df['query']]

In [ ]:
%%timeit
df['word_count'] = df['query'].str.count(r'\s') +1

## Plots mit neuen Features
Mit dem aktuellen DataFrame werden wir nun folgende Plots erstellen:
- Zeitverlauf mit Brand und Nonbrand
- Ein Histogram der Seiten für Klicks
- Einen Stripplot der Klicks verteilt auf die Anzahl der Wörter

In [ ]:
df.groupby(['date','brand'], as_index=False)['clicks'].sum() \
    .pipe((sns.lineplot, 'data'), x='date', y='clicks', hue='brand')

In [ ]:
ax = df.query('(clicks > 100) and ~(page.isin(["https://www.serienjunkies.de/"]))') \
    .groupby('page')[['clicks']].sum() \
    .pipe(sns.distplot, kde=False)
ax.set_xlabel('Klicks')
ax.set_ylabel('Frequenz')
# .query('(clicks > 100) and ~(page.isin(["https://www.serienjunkies.de/"]))') \

In [ ]:
df.groupby('page')[['clicks']].sum().describe()

In [ ]:
df.query('brand == False and page != "https://www.serienjunkies.de/"') \
    .groupby(['word_count', 'query'], as_index=False).sum() \
    .pipe((sns.stripplot, 'data'), x='word_count', y='clicks')

# .query('brand == False and page != "https://www.serienjunkies.de/"') \

## Abspeichern mit Dataset
Die geladenen Daten können natürlich auch abgespeichert werden. Eine Datenbank ist da immer sinnvoll. In unseren simplen Beispielen werden wir mit einer einfachen Sqlite arbeiten. Um es noch einfacher zu machen, verwenden wir das Package `dataset` [(link)](https://dataset.readthedocs.io/en/latest/).

In [ ]:
import dataset

In [ ]:
db = dataset.connect('sqlite:///data/campixx_sa.db')

In [ ]:
df.to_sql('searchanalytics_data', con=db.engine)

### CSV Datein, Excel und weitere Formate sind natürlich möglich.
Alle Funktionen für Input / Output gibt's [hier](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)
```python
# csv
df.to_csv('data/searchanalytics_data.csv')
# excel
df.to_excel(...)
# google big query
df.to_gbq(...)
# viele weitere
df.to + Tab drücken!
```

# Searchanalytics Daten speichern
Da die Daten der GSC auf 16 Monate beschränkt sind, macht es durchaus Sinn sie abzuspeichern. Für alle **SEOs** ist das zumindest eine **Pflicht**. Wie wir eben gesehen haben, ist das eigentlich nicht so schwierig. 
Um die Daten in größerem Umfang zu speichern, muss man allerdings mindestens folgendes tun:
- Die Daten müssen für jeden Tag einzeln abgefragt werden. *(jeder Tag = eine Abfrage)*
    - Pro Tag können maximal 50.000 Zeilen in der Antwort enthalten sein.
- Die Abfragen müssen für die möglichen Kombinationen der Dimensionen durchgeführt werden. *[country, device, page, query]*
    - Wir haben ja festgestellt, dass die Mengen sich je nach Dimensionen unterscheiden.
- Wenn ich die Daten zur searchAppearance haben möchte, muss ich über alle möglichen Filter iterieren.
    - searchAppearance kann mit keiner anderen Dimension kombiniert werden.
- Für verschiedene Google Accounts, muss ich mehrere Auth-Files speichern und den Abfragen zuweisen.
- Man muss sich merken, welche Tage bereits abgefragt wurden und welche nicht.
    - Wenn während der Abfrage die API nicht mehr antwortet, muss man von neuem anfangen.
- Man sollte ebenfalls prüfen, an welchen Tagen Daten vorliegen, um die API quotas nicht zu verschwenden.

**Also wie immer:** *"mal eben schnell"* ist **immer** weder *mal eben*, noch *schnell*...

![mal eben schnell scripten](data/mal_eben.jpg)

# Für die Campixx hab ich daher was gebaut
**Disclaimer:** Nur für diejenigen, die es nicht selbst können. 
- Daten werden "nur" in eine lokale Sqlite gespeichert
- Daten werden nicht normalisiert
- Zum arbeiten (32 GB Daten) hat alles gut funktioniert 😝

→ [gsc-sa-downloader auf GitHub](https://github.com/Jonnyblacklabel/gsc_sa_downloader)

## So geht's
Readme auf GitHub lesen und den Anweisungen zur Installation folgen.

## Ihr habt `pipenv install` ausgeführt?
Alles weitere läuft per CLI. Also öffnet eine `cmd` (oder PowerShell, Terminal, etc...) in dem Ordner in den ihr alles kopiert habt.
```
# Download der GSC Daten für eure Property
pipenv shell
cd gsc_sa_downloader
python gsc_sa_downloader.py download [account name] [property mit trailing slash "/"] --generate
```
Da der Account verifiziert werden muss, kommt der übliche Prozess in eurem Browser.
Danach sollte es auch schon losgehen. Ich hoffe es funktioniert 😏